# Session 12

[![Open and Execute in Google Colaboratory](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/astrojuanlu/ie-mbd-python-data-analysis-i/blob/main/sessions/Session%2012.ipynb)

- The index in pandas
- `.loc` vs `.iloc`
- Merge & Join
- Dealing with missing data

## The index in pandas

> There is just one core concept that brings together almost all of the pandas API [...]: the index and index alignment.

James Powell ([source](https://youtu.be/pjq3QOxl9Ok?si=B-wOGZJ7XvO70zmk))

The index is a property of pandas DataFrames that allows you to refer to specific rows by _label_, rather than by _position_. You have already used it, but there is so much that can be done with it.

In [ ]:
df_covid = pd.read_csv(
    "https://github.com/astrojuanlu/ie-mbd-python-data-analysis-i/raw/main/"
    "data/national_covid19.csv"
)
df_covid.head()

You can visually distinguish the index on the left hand side because the values are highlighted in bold. By default, an autoincrement, integer index is used, like in this case.

Remember that you can use `.loc` to index and slice a DataFrame by label:

In [ ]:
df_covid.loc[0:3]

However, things start to get interesting when you use some other column as an index. In this case, let's use the date:

In [ ]:
df_covid["date"].is_unique  # Good sanity check, not mandatory

In [ ]:
df_covid_r = df_covid.set_index("date")
df_covid_r.head()

Notice that `date` is no longer a column!

In [ ]:
df_covid_r["date"]  # Fails

But more importantly, now you can use the date as the index:

In [ ]:
df_covid_r.loc["2020-04-01":"2020-04-05"]

## `.loc` vs `.iloc`

Now, `.loc` is very powerful because it allows you to use labels, rather than positions. But what if you want positions for some reason?

Well, that's what `.iloc` is for:

In [ ]:
df_covid_r.iloc[:3]

Notice that `.iloc` follows the Python semantics of slicing (the end is not included), unlike `.loc`:

In [ ]:
df_covid.loc[:3]

## Exercises

### 1. Wildfires in Spain

Read the `data/fires-all.csv` file into a pandas DataFrame.

How many fires were there in the year 2018? Compute that in 2 ways:
- Filtering on the `fecha` column
- Setting `fecha` as the index

Verify that the result is exactly the same.

In [ ]:
FIRES_URL = (
    "https://github.com/astrojuanlu/ie-mbd-python-data-analysis-i/raw/main/"
    "data/fires-subset.csv"
)
FIRES_URL

## Merge & Join

Concatenating DataFrames makes sense for "partitioned" data:
- Identical schema, different rows (`concat`)
- Identical rows, additional columns (`concat(axis="columns")`)

However, many more situations require a smarter way of combining datasets.

The `.merge` method implements SQL `JOIN`-like operations in pandas.

(`.join` is similar to `.merge`, but includes some optimizations when working with indexes)

In [ ]:
import pandas as pd

Use case: we have our big renters data for Madrid, with lots of rich information:

In [ ]:
df_madrid = pd.read_csv(
    "https://github.com/astrojuanlu/ie-mbd-python-data-analysis-i/raw/main/"
    "data/grandes-tenedores-madrid.csv"
)
df_madrid.head()

And we have data for Spain as well, but it's very lacking:

In [ ]:
df_spain = pd.read_csv(
    "https://github.com/astrojuanlu/ie-mbd-python-data-analysis-i/raw/main/"
    "data/megatenedores_estatal_2024.csv"
)
df_spain.head()

Would it be possible to _augment_ the Spain data with part of the information coming from the Madrid one?

In [ ]:
df_madrid_company_data = (
    df_madrid.loc[:, ["NIF", "Filial propietaria directa", "Matriz"]]
    .rename(columns={
        "Matriz": "Holding",
        "Filial propietaria directa": "Branch",
    })
)
df_madrid_company_data.head()

In [ ]:
len(df_madrid_company_data)

Both DataFrames have the `Matriz` column ("Holding") in common. However, for each `Matriz`, there is more than one possible value of `Filial` ("Branch"):

In [ ]:
df_madrid_company_data["Holding"].nunique()

In [ ]:
(
    df_madrid_company_data
    .groupby("Holding")["Branch"].size()
    .sort_values()
)

In [ ]:
(
    df_madrid_company_data
    .loc[df_madrid_company_data["Holding"] == "Blackstone"]
    .head()
)

For the purposes of this exercise, let's pretend that `1 Holding = 1 Branch`.

In [ ]:
df_madrid_company_data_simple = (
    df_madrid_company_data.drop_duplicates(subset="Holding")
)
df_madrid_company_data_simple["Branch"].is_unique

Now, combining this data is a matter of calling the `.merge` method appropriatly:

In [ ]:
df_spain_augmented = (
    df_spain
    .merge(
        df_madrid_company_data_simple,
        how="left",  # same as `LEFT JOIN` in SQL
        left_on="Matriz",
        right_on="Holding",
    )
)
df_spain_augmented.head()

Notice that, where there was available data, pandas combined the rows, leaving `NaN` in the rest of the places.

The `.join` method is like the `.merge` one, but it's more efficient when working with indexes.

In [ ]:
df_madrid_company_data_simple.head(1)

In [ ]:
df_spain.head(1)

In [ ]:
df_spain.set_index("Matriz").head(1)

In [ ]:
(
    df_spain.set_index("Matriz")
    .join(
        df_madrid_company_data_simple.set_index("Holding")
    )
    .head()
)

## Exercises

### 2. European Commission lobbists

Below you can find some data coming from Civio on lobby meetings at the European Commission.

In [ ]:
LOBBY_DATA_URL = (
    "https://github.com/astrojuanlu/ie-mbd-python-data-analysis-i/raw/main/data/commission-lobbists-meetings.csv"
)

However, we know very little about the "lobbyst". Below, you can find some extra information (AI-generated) on some of them.

In [ ]:
LOBBY_DATA_AI_GEN_URL = (
    "https://github.com/astrojuanlu/ie-mbd-python-data-analysis-i/raw/main/data/"
    "lobby_data_ai.csv"
)

Combine both datasets to answer these questions:

- What are the top 10 lobbysts in terms of number of meetings?
- How many meetings happened with US vs with EU-based lobbysts?
- What was the most common policy orientation of the meetings?
- Now, answer the same questions, but using only meetings with a "high" number of representatives (let's use `nr > mean(nr)`)

## Dealing with missing data

As you have noticed already, lots of pandas operations generate missing data in the form of `NaN`.

Filtering this data with the usual methods doesn't work, because, following the IEEE 754 standard, NaN is different from itself:

In [ ]:
float("nan") == float("nan")

pandas DataFrames have special methods to deal with missing data:

In [ ]:
df_spain_augmented.head()

Filter:

In [ ]:
df_spain_augmented.loc[
    df_spain_augmented["NIF"].isna()
].head()

Dropping rows with missing data:

In [ ]:
df_spain_augmented.dropna(subset="NIF").head()

Filling missing data:

In [ ]:
df_spain_augmented.fillna({"NIF": "<UNKNOWN>"}).head()

## Exercises

### 3. Missing location and puzzling columns

- Observe that, in the European Commission dataset, the `location` column is sometimes null. Inspect how many rows have this property. What would you do with those? (Open question)
- There are some trailing columns in the dataset with some null values. How many non-null values do they have? What would you do with those? (Open question with a "more correct" answer)